In [94]:
import os
import pandas as pd
import numpy as np
from mailparser import parse_from_string, parse_from_bytes
import re
import nltk


In [104]:
data = pd.read_csv("enron_emails.csv")
data.head(5)


,Unnamed: 0,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
0,0,<29790972.1075855665306.JavaMail.evans@thyme>,2000-12-13 18:41:00-08:00,1.11913372.-2@multexinvestornetwork.com,pallen@enron.com,"December 14, 2000 - Bear Stearns' predictions ...",\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,In today's Daily Update you'll find free repor...,8486,0,0
1,1,<21975671.1075855665520.JavaMail.evans@thyme>,2000-12-13 08:35:00-08:00,messenger@ecm.bloomberg.com,NaN,Bloomberg Power Lines Report,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,Here is today's copy of Bloomberg Power Lines....,16252,0,0
2,2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
3,3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
4,4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,Allen-P,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0,0


In [105]:
data.drop(columns=data.columns[0], axis=1, inplace=True)


In [106]:
# remove any obvious spams/newsletters
data = data[~data['Content'].str.contains('Copyright')]
data = data[~data['Content'].str.contains('ADVERTISEMENT')]
data = data[~data['Content'].str.contains('Advertisement')]
data = data[~data['Content'].str.contains('advertisement')]
data = data[~data['Content'].str.contains('Click Here')]
data = data[~data['Content'].str.contains('unsubscribe')]
data = data[~data['Content'].str.contains('UNSUBSCRIBE')]
data = data[~data['Content'].str.contains('e-newsletter')]
data = data[~data['Content'].str.contains('(tm)')]



C:\Users\Ryan\AppData\Local\Temp\ipykernel_20968\601240462.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data = data[~data['Content'].str.contains('(tm)')]


In [107]:
data.shape


(180486, 12)

In [102]:
data

,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0,0
5,<14670081.1075855667751.JavaMail.evans@thyme>,2000-10-05 05:55:00-07:00,phillip.allen@enron.com,paula.harris@enron.com,Re: 2001 Margin Plan,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,"Paula,\n\n 35 million is fine\n\nPhillip",36,1,0
6,<2101427.1075855667773.JavaMail.evans@thyme>,2000-10-04 09:23:00-07:00,phillip.allen@enron.com,ina.rangel@enron.com,"Var, Reporting and Resources Meeting",\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,969,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
224102,<26807948.1075842029936.JavaMail.evans@thyme>,2001-11-28 13:30:11-08:00,john.zufferli@enron.com,kori.loibl@enron.com,Trade with John Lavorato,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,This is a trade with OIL-SPEC-HEDGE-NG (John L...,341,0,0
224103,<25835861.1075842029959.JavaMail.evans@thyme>,2001-11-28 12:47:48-08:00,john.zufferli@enron.com,john.lavorato@enron.com,Gas Hedges,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,Some of my position is with the Alberta Term b...,117,0,0
224104,<28979867.1075842029988.JavaMail.evans@thyme>,2001-11-28 07:20:00-08:00,john.zufferli@enron.com,dawn.doucet@enron.com,RE: CONFIDENTIAL,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,2\n\n -----Original Message-----\nFrom: \tDouc...,326,0,0
224105,<22052556.1075842030013.JavaMail.evans@thyme>,2001-11-27 11:52:45-08:00,john.zufferli@enron.com,jeanie.slone@enron.com,Calgary Analyst/Associate,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,170,0,0


In [10]:
data['Origin'] = data['Origin'].convert_dtypes("str")
data['Origin'] = data['Origin'].str.lower()
data['Origin'].dtype


string[python]

In [ ]:
# number of unique individuals
data['Origin'].nunique()


158

In [111]:
execs = pd.read_csv("POI.csv", names=["last", "first", "j"])
execs["Origin"] = execs['last'] + "-" + execs['first'].astype(str).str[0]
execs["title"] = "executive"
execs = execs.drop(columns=["last", "first", "j"])



In [113]:
execs['Origin'] = execs['Origin'].str.lower()
# execs = execs.set_index(execs.columns[0]).T.to_dict('records')
execs = dict(execs.values)


TypeError: list indices must be integers or slices, not str

In [49]:
execs

{'lay-k': 'executive',
 'skilling-j': 'executive',
 'yeager-s': 'executive',
 'hirko-j': 'executive',
 'shelby-r': 'executive',
 'causey-r': 'executive',
 'calger-c': 'executive',
 'hannon-k': 'executive',
 'koenig-m': 'executive',
 'forney-j': 'executive',
 'rice-k': 'executive',
 'rieker-p': 'executive',
 'fastow-l': 'executive',
 'fastow-a': 'executive',
 'delainey-d': 'executive',
 'glisan-b': 'executive',
 'belden-t': 'executive',
 'kopper-m': 'executive',
 'bowen-r': 'executive',
 'colwell-w': 'executive'}

In [77]:
e = data.copy()

In [75]:
# e['title'] = np.NaN

In [79]:
keep_nan = [k for k,v in execs.items() if pd.isnull(v)]
s = e['Origin']

e['title'] = s.map(execs).fillna(s.mask(s.isin(keep_nan)))

In [117]:
for emp in e['title']:
    if emp != "executive":
        emp == "employee"
    

In [13]:
exec_list = execs.index.to_list()
exec_list


['lay-k',
 'skilling-j',
 'yeager-s',
 'hirko-j',
 'shelby-r',
 'causey-r',
 'calger-c',
 'hannon-k',
 'koenig-m',
 'forney-j',
 'rice-k',
 'rieker-p',
 'fastow-l',
 'fastow-a',
 'delainey-d',
 'glisan-b',
 'belden-t',
 'kopper-m',
 'bowen-r',
 'colwell-w']

In [14]:
data.shape


(180486, 12)

In [15]:
clean_data = data.copy()


In [16]:
clean_data


,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0,0
5,<14670081.1075855667751.JavaMail.evans@thyme>,2000-10-05 05:55:00-07:00,phillip.allen@enron.com,paula.harris@enron.com,Re: 2001 Margin Plan,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,"Paula,\n\n 35 million is fine\n\nPhillip",36,1,0
6,<2101427.1075855667773.JavaMail.evans@thyme>,2000-10-04 09:23:00-07:00,phillip.allen@enron.com,ina.rangel@enron.com,"Var, Reporting and Resources Meeting",\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,969,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
224102,<26807948.1075842029936.JavaMail.evans@thyme>,2001-11-28 13:30:11-08:00,john.zufferli@enron.com,kori.loibl@enron.com,Trade with John Lavorato,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,This is a trade with OIL-SPEC-HEDGE-NG (John L...,341,0,0
224103,<25835861.1075842029959.JavaMail.evans@thyme>,2001-11-28 12:47:48-08:00,john.zufferli@enron.com,john.lavorato@enron.com,Gas Hedges,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,Some of my position is with the Alberta Term b...,117,0,0
224104,<28979867.1075842029988.JavaMail.evans@thyme>,2001-11-28 07:20:00-08:00,john.zufferli@enron.com,dawn.doucet@enron.com,RE: CONFIDENTIAL,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,2\n\n -----Original Message-----\nFrom: \tDouc...,326,0,0
224105,<22052556.1075842030013.JavaMail.evans@thyme>,2001-11-27 11:52:45-08:00,john.zufferli@enron.com,jeanie.slone@enron.com,Calgary Analyst/Associate,"\ExMerge - Zufferli, John\Sent Items",zufferli-j,john zufferli 6-26-02.PST,Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...,170,0,0


In [17]:
# checking to see if any employee names were dropped when dropping null email addresses
clean_data['Origin'].nunique()


158

In [18]:
clean_data.shape


(180486, 12)

In [ ]:
# emails sent and recieved internally
# to_enron = clean_data[clean_data['To'].str.contains('enron.com')]
# enron = to_enron[to_enron['From'].str.contains('enron.com')]


In [84]:
clean_data.head()


,Message-ID,Date,From,To,Subject,Path,Origin,FileName,Content,Content Length,Reply,Jail
2,<7452188.1075855667684.JavaMail.evans@thyme>,2000-10-09 07:16:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
3,<23790115.1075855667708.JavaMail.evans@thyme>,2000-10-09 07:00:00-07:00,phillip.allen@enron.com,keith.holst@enron.com,Consolidated positions: Issues & To Do list,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,5688,0,0
4,<5860470.1075855667730.JavaMail.evans@thyme>,2000-10-05 06:26:00-07:00,phillip.allen@enron.com,david.delainey@enron.com,NaN,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,"Dave, \n\n Here are the names of the west desk...",120,0,0
5,<14670081.1075855667751.JavaMail.evans@thyme>,2000-10-05 05:55:00-07:00,phillip.allen@enron.com,paula.harris@enron.com,Re: 2001 Margin Plan,\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,"Paula,\n\n 35 million is fine\n\nPhillip",36,1,0
6,<2101427.1075855667773.JavaMail.evans@thyme>,2000-10-04 09:23:00-07:00,phillip.allen@enron.com,ina.rangel@enron.com,"Var, Reporting and Resources Meeting",\Phillip_Allen_Dec2000\Notes Folders\All docum...,allen-p,pallen.nsf,---------------------- Forwarded by Phillip K ...,969,0,0


In [20]:
# search email bodies for Arther Andersen related content
# *both anderson and andersen are used in the email bodies and in email handles - @us.arthurandersen.com and @uk.arthuranderson.com
accounting = ["arthur andersen", "Arthur Andersen", "Arthur Anderson", "arther anderson", "artherandersen",
              "arthuranderson", "Nancy", "Nancy Temple", "David Duncan", "Duncan", "david duncan", "AA"]


def filter_subgroup(df, ls, filter):
    """
    Args:
        df (_type_): _description_
        ls (1D list): list of filter words/phrases
        filter (str): options: content(email content), poi(originating inbox formatted "lastname-first_initial" in lowercase)

    Returns:
        filtered_df
    """
    aa_id = []
    aa_corpus = []
    to_ls = []
    recipient_ls = []
    poi_ls = []
    filt = 0
    
    
    if filter == "content":
        filt += 1
        
    for a in ls:
        for body, id, to, rec, poi in zip(df['Content'], df['Message-ID'], df['To'], df['From'], df['Origin']):

            if filt == 1: 
                column = str(body)
            else:
                column = str(poi)

    
            x = re.search(a, column)
        

            if x:
                aa_id.append(id)
                to_ls.append(to)
                aa_corpus.append(body)
                recipient_ls.append(rec)
                poi_ls.append(poi)

            else:
                pass

    filtered_df = pd.DataFrame(list(zip(aa_id, poi_ls, recipient_ls, to_ls, aa_corpus)), columns=[
                               "Id", "Poi", "From", "To", "Content"])

    return filtered_df


a_anderson_df = filter_subgroup(clean_data,accounting,filter="content")
executive_inbox = filter_subgroup(clean_data, exec_list, filter="poi")

In [ ]:
print(a_anderson_df.shape)
print(executive_inbox.shape)

(3319, 5)
(4205, 5)


In [ ]:
a_anderson_df.to_csv("temp/arthur_andersen_filtered.csv")
clean_data.to_csv('temp/general_emails.csv')
executive_inbox.to_csv('temp/executives_inboxes.csv')

In [21]:
print(f"arthurandersen: {a_anderson_df.shape}")
print(f"general emails: {clean_data.shape}")
print(f"from/to POIs: {executive_inbox.shape}")


arthurandersen: (3383, 5)
general emails: (180486, 12)
from/to POIs: (4372, 5)


In [93]:
clean_data['Content'].view()

2         ---------------------- Forwarded by Phillip K ...
3         ---------------------- Forwarded by Phillip K ...
4         Dave, \n\n Here are the names of the west desk...
5                  Paula,\n\n 35 million is fine\n\nPhillip
6         ---------------------- Forwarded by Phillip K ...
                                ...                        
224102    This is a trade with OIL-SPEC-HEDGE-NG (John L...
224103    Some of my position is with the Alberta Term b...
224104    2\n\n -----Original Message-----\nFrom: \tDouc...
224105    Analyst\t\t\t\t\tRank\n\nStephane Brodeur\t\t\...
224106    i think the YMCA has a class that is for peopl...
Name: Content, Length: 180486, dtype: object